In [1]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

c:\Users\saiyu\Desktop\projects\AWS\AI_ML_OPS\Housing_Regression_ML_EndtoEnd\.venv\Scripts\python.exe
3.0.4
c:\Users\saiyu\Desktop\projects\AWS\AI_ML_OPS\Housing_Regression_ML_EndtoEnd\.venv\Lib\site-packages\xgboost\__init__.py


In [3]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

c:\Users\saiyu\Desktop\projects\AWS\AI_ML_OPS\Housing_Regression_ML_EndtoEnd\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("../data/processed/feature_engineered_train.csv")
eval_df  = pd.read_csv("../data/processed/feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (585244, 39)
Eval shape: (149424, 39)


In [7]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [9]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("../mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2025/10/31 14:48:37 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-10-31 14:48:37,127] A new study created in memory with name: no-name-68d4a3c7-228b-4a1a-8228-b48df3fddb31
[I 2025-10-31 14:48:44,508] Trial 0 finished with value: 77487.90590691846 and parameters: {'n_estimators': 214, 'max_depth': 4, 'learning_rate': 0.10407732316482468, 'subsample': 0.5053771387867868, 'colsample_bytree': 0.9236190867314102, 'min_child_weight': 1, 'gamma': 3.600196746321478, 'reg_alpha': 0.007414504148023852, 'reg_lambda': 2.534671129163125e-08}. Best is trial 0 with value: 77487.90590691846.
[I 2025-10-31 14:49:13,580] Trial 1 finished with value: 85494.3045042292 and parameters: {'n_estimators': 868, 'max_depth': 5, 'learning_rate': 0.27412810639981244, 'subsample': 0.8262983802457371, 'colsample_bytree': 0.5877555700046563, 'min_child_weight': 7, 'gamma': 1.3973973854567834, 'reg_alpha': 5.884890705228703, 'reg_lambda': 

Best params: {'n_estimators': 985, 'max_depth': 5, 'learning_rate': 0.08967605575745337, 'subsample': 0.7462115474065567, 'colsample_bytree': 0.5299286740518465, 'min_child_weight': 1, 'gamma': 0.7479908322622142, 'reg_alpha': 3.994636009640168e-07, 'reg_lambda': 0.0008077467251309673}


In [7]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 31472.002185832847
RMSE: 70459.0823185452
R²: 0.9616351065691122


/Users/riadanas/Desktop/housing regression MLE/.venv/lib/python3.11/site-packages/xgboost/sklearn.py:1028: UserWarning: [13:59:53] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/09/23 13:59:56 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/09/23 13:59:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
